In [ ]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager

In [ ]:
def load_iceberg_raw_eod_yfinance(symbol_date_pairs, iceberg_sink_table, schema_config_file):
    try: 
        table_manager=TableManager(schema_config_file)
        schema_struct_type=table_manager.get_struct_type(iceberg_sink_table)   

        # print(schema_struct_type)
        create_table_script = table_manager.get_create_table_query(iceberg_sink_table)
        spark.sql(create_table_script)

        df_source=parallel_fetch_yfinance_record(symbol_date_pairs, schema_struct_type)
        
        # df_source.writeTo(iceberg_sink_table).append()
        df_source.write.mode("overwrite").saveAsTable(iceberg_sink_table)    
        # df_source.writeTo(iceberg_sink_table).overwritePartitions() 

        print(f"{iceberg_sink_table} has been loaded")
        
    except Exception as e:
        print(f"Error loading lceberg raw table: {e}")
        

In [ ]:
def load_pg_finalytics(iceberg_source_table, pg_url, pg_driver, pg_sink_table, pg_truncate_script, pg_merge_script):   
    try:     
        df_source=spark.read.table(iceberg_source_table)          
        finalytics.execute_sql_script(pg_truncate_script)
        
        # Write DataFrame to PostgreSQL
        df_source.write.jdbc(url=pg_url, table=pg_sink_table, mode="append", properties={"driver": pg_driver})        
        finalytics.execute_sql_script(pg_merge_script)
    except Exception as e:
        print(f"Error loading pg finalytics: {e}")        


## Load nessie.raw.stock_eod_yfinance

In [ ]:
# symbol_start_date_pairs = [
#     ('TSLA', '2024-11-20'),
#     ('AAPL', '2024-11-20'),
#     ('C', '2024-11-20'),
# ]

# Get finalytics connetion info
conn_config_file='cfg_connections.yaml'
finalytics=PGDatabaseManager(conn_config_file, 'finalytics')
pg_url=finalytics.jdbc_url
pg_driver=finalytics.driver

# Get symbol_start_date_pairs from finalytics
query="select symbol, start_date from fin.vw_etl_stock_eod_start_date limit 20"
# query="select symbol, start_date from fin.vw_etl_stock_eod_start_date"
symbol_start_date_pairs=finalytics.get_sql_script_result_list(query)
# print(symbol_start_date_pairs)


# Get iceberg table config info
table_schema_config_file='cfg_table_schemas.yaml'
iceberg_table='nessie.raw.stock_eod_yfinance'

# Set global import_time
import_time = datetime.now().isoformat()

# Load nessie.raw.stock_eod_yfinance 
load_iceberg_raw_eod_yfinance(symbol_start_date_pairs, iceberg_table, table_schema_config_file)


## Load finalytics.stage.stock_eod_quote_yahoo and merge into fin.stock_eod_quote

In [ ]:
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

pg_truncate_script = f"truncate table {pg_table};"

pg_merge_script = "call fin.usp_load_stock_eod();"

load_pg_finalytics(iceberg_table, pg_url, pg_driver, pg_table,  pg_truncate_script, pg_merge_script)

In [ ]:
spark.sql(f'select count(*) from {iceberg_table}').show()

In [ ]:
spark.sql('select * from nessie.raw.stock_eod_yfinance').show(truncate=False)

In [ ]:
# print(symbol_start_date_pairs)